# GANGANSTYLE

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import zipfile
from PIL import Image

In [ ]:
dimension = 64
initial_g_dimension = 8
random_size = 16*16

In [ ]:
def openImages(path):
    archive = zipfile.ZipFile(path, 'r')
    np_im_array = []

    for name in archive.namelist():
        imgfile = archive.open(name)
        img = Image.open(imgfile)
        np_im_array.append(np.array(img).reshape(dimension * dimension) / 255)

    return np_im_array


In [ ]:
def next_batch(images, batch_size):
    indices = np.random.randint(0, len(images), batch_size)
    batch_images = []
    for i in indices:
        batch_images.append(( images[i].reshape(dimension * dimension) ) * 2 - 1  )

    return batch_images

In [ ]:
fake_fg_images = openImages('C:/resources/fake_photos_jpg.zip')
real_fg_images = openImages('C:/resources/real_photos_jpg.zip')

## The Generator

In [ ]:
def generator(z,reuse=None):
    with tf.variable_scope('gen',reuse=reuse):
        alpha = 0.2

        hidden1 = tf.layers.dense(inputs=z,units=8*8*128)
        hidden1 = tf.maximum(alpha*hidden1,hidden1)        
        reshaped_hidden1 = tf.reshape(hidden1,[-1,8,8,128] )  
        
        convo_1 = tf.layers.conv2d_transpose(inputs=reshaped_hidden1,filters=128,kernel_size=5,padding="same", strides=2)
        convo_1 = tf.maximum(alpha*convo_1,convo_1)

        convo_2 = tf.layers.conv2d_transpose(inputs=convo_1,filters=128,kernel_size=5,padding="same", strides=2)
        convo_2 = tf.maximum(alpha*convo_2,convo_2)
        
        convo_3 = tf.layers.conv2d_transpose(inputs=convo_2,filters=128,kernel_size=5,padding="same", strides=2)
        convo_3 = tf.maximum(alpha*convo_3,convo_3)
        
        convo_4 = tf.layers.conv2d(inputs=convo_3,filters=1,kernel_size=5,padding="same", strides=1,activation=tf.nn.tanh)
        
        output = tf.reshape(convo_4,[-1,dimension*dimension] )
        
        return output

## The Discriminator

In [ ]:
def discriminator(X,reuse=None):
    with tf.variable_scope('dis',reuse=reuse):
        alpha = 0.2
        
        reshaped_X = tf.reshape(X,[-1,dimension,dimension,1] )        
        convo_1 = tf.layers.conv2d(inputs=reshaped_X,filters=64,kernel_size=5,padding="same", strides=2)
        convo_1 = tf.maximum(alpha*convo_1,convo_1)
        
        convo_2 = tf.layers.conv2d(inputs=convo_1,filters=64,kernel_size=5,padding="same", strides=2)
        convo_2 = tf.maximum(alpha*convo_2,convo_2)
        
        convo_3 = tf.layers.conv2d(inputs=convo_2,filters=64,kernel_size=5,padding="same", strides=2)
        convo_3 = tf.maximum(alpha*convo_3,convo_3)        
        convo_3_flat = tf.reshape(convo_3,[-1,8*8*64])  
        
        logits = tf.layers.dense(convo_3_flat,units=1)
        output = tf.sigmoid(logits)
    
        return logits, output

### Placeholders

In [ ]:
real_images = tf.placeholder(tf.float32,shape=[None,dimension*dimension])
noise_images = tf.placeholder(tf.float32,shape=[None,random_size])

### Generator

In [ ]:
G_model = generator(noise_images)

### Discriminator 

In [ ]:
D_logits_real, D_output_real = discriminator(real_images)

In [ ]:
D_logits_fake, D_output_fake = discriminator(G_model,reuse=True)

### Losses

In [ ]:
disc_learning_rate = 1e-4
gen_learning_rate = 1e-4

In [ ]:

d_loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_real, labels=tf.ones_like(D_logits_real)) +
    tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_fake, labels=tf.zeros_like(D_logits_fake)))

g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_fake, labels=tf.ones_like(D_logits_fake)))


In [ ]:

gen_vars = tf.trainable_variables('gen' )
disc_vars = tf.trainable_variables('dis') 

D_trainer = tf.train.AdamOptimizer(disc_learning_rate, beta1=0.5).minimize(d_loss, var_list=disc_vars)
G_trainer = tf.train.AdamOptimizer(gen_learning_rate, beta1=0.5).minimize(g_loss, var_list=gen_vars)

## Training Session

In [ ]:
batch_size = 128
epochs = 1500
init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=gen_vars)

In [ ]:
with tf.Session() as sess:
    
    sess.run(init)
    
    for e in range(epochs):

        num_batches = len(real_fg_images) // batch_size
        
        start_time = time.time()
        
        for i in range(num_batches):
            

            batch_images = next_batch( real_fg_images, batch_size )
            batch_z = np.random.uniform(-1, 1, size=(batch_size, random_size))
            
            _ = sess.run(D_trainer, feed_dict={real_images: batch_images, noise_images: batch_z})
            _ = sess.run(G_trainer, feed_dict={real_images: batch_images, noise_images: batch_z})       
        
        end_time = time.time()
        
        print("Epoch: {} ".format(e + 1))
        print("Time: {} ".format(end_time - start_time) )
        
        if e % 10 == 0:
            
            noise_sample = np.random.uniform(-1, 1, size=(20, random_size))
            gen_sample = generator(noise_images ,reuse=True).eval(feed_dict={noise_images: noise_sample})
            fig=plt.figure(figsize=(10, 10))
            
            for i in range(1,20):
                reshaped_part = gen_sample[i]
                fig.add_subplot(5, 5, i)
                plt.imshow(reshaped_part.reshape(dimension, dimension),cmap='Greys')
            plt.show()
        
        saver.save(sess, './models/ganganstyle.ckpt')

In [ ]:
with tf.Session() as sess:

    saver.restore(sess,'./models/ganganstyle.ckpt' )
    noise_sample = np.random.uniform(-1, 1, size=(20, random_size))
    gen_sample_fake = sess.run(generator(noise_images ,reuse=True),feed_dict={noise_images: noise_sample})
    fig=plt.figure(figsize=(10, 10))
    for i in range(1,20):
        reshaped_part = gen_sample_fake[i]
        fig.add_subplot(5, 5, i)
        plt.imshow(reshaped_part.reshape(dimension, dimension),cmap='Greys')

In [ ]:
plt.imshow(gen_sample_fake[0].reshape(dimension,dimension), cmap='Greys')